In [1]:
# import
import os
import numpy as np

# import plotting libraries
import matplotlib.pyplot as plt
plt.rcParams.update({'font.size': 10})
plt.rcParams['svg.fonttype'] = 'none'

# import network_control functions
from network_control.energies import integrate_u, get_control_inputs
from network_control.utils import matrix_normalization, convert_states_str2int, normalize_state

# Using get_control_inputs

This notebook covers examples of different ways in which get_control_inputs can used to derive control energy

### Load A

In [2]:
# directory where data is stored
datadir = '/Users/lindenmp/Google-Drive-Penn/work/research_projects/control_package/data'
resultsdir = '/Users/lindenmp/Google-Drive-Penn/work/research_projects/control_package/results'
A_file = 'pnc_schaefer200_Am.npy'

# load A matrix
A = np.load(os.path.join(datadir, A_file))
n_nodes = A.shape[0]
print(A.shape)

(200, 200)


### Get initial and target state

In [3]:
# load node-to-system mapping
system_labels = list(np.loadtxt(os.path.join(datadir, 'pnc_schaefer200_system_labels.txt'),
                                dtype=str))

# use list of system names to create states
states, state_labels = convert_states_str2int(system_labels)

# extract initial state
x0 = states == state_labels.index('Vis')

# extract target state
xf = states == state_labels.index('Default')

# normalize state magnitude
x0 = normalize_state(x0)
xf = normalize_state(xf)

# Get energy

By default, we compute $x$ and $u$ using the following settings:
1. Time system: continuous (*system*)
2. Time horizon: 1 (*T*)
3. Control set: uniform full (*B*)
4. Amplitude of $x$ and $u$ are constrained toward zero (*xr*)
5. The constraints on $x$ are equal to the constraints on $u$ (*rho*)
6. The constraints on $x$ are applied to all system nodes (*S*)

Note, terms in brackets represent the variables that govern each of these settings

You can compute $x$ and $u$ using the above settings like so:

In [4]:
system = 'continuous'  # choose time system
A_norm = matrix_normalization(A, system=system, c=1)  # normalize A
B = np.eye(n_nodes)  # uniform full control set
T = 1  # time horizon
xr = np.zeros(n_nodes)  # x and u constrained toward zero activity
rho = 1  # x and u constrained equally
S = np.eye(n_nodes)  # x is constrained over all nodes

# get the state trajectory (x) and the control inputs (u)
x, u, n_err = get_control_inputs(A_norm=A_norm, T=T, B=B, x0=x0, xf=xf, system=system, xr=xr, rho=rho, S=S)
node_energy = integrate_u(u)
energy = np.sum(node_energy)

print('energy: {0:.2f}'.format(energy))

energy: 2604.71


## Variations

All of the below code cells represent variations to the above default settings. Changes are noted in comments.

### You want to relax the constraints on $x$?

Note, constraints on $u$ cannot be change; the control signals are always minimized in the cost function

In [5]:
system = 'continuous'  # choose time system
A_norm = matrix_normalization(A, system=system, c=1)  # normalize A
B = np.eye(n_nodes)  # uniform full control set
T = 1  # time horizon
xr = np.zeros(n_nodes)  # x and u constrained toward zero activity
rho = 0.5  # x constrained less than u  <-- NOTE CHANGE.
S = np.eye(n_nodes)  # x is constrained over all nodes

# get the state trajectory (x) and the control inputs (u)
x, u, n_err = get_control_inputs(A_norm=A_norm, T=T, B=B, x0=x0, xf=xf, system=system, xr=xr, rho=rho, S=S)
node_energy = integrate_u(u)
energy = np.sum(node_energy)

print('energy: {0:.2f}'.format(energy))

energy: 2686.91


### You want to only constrain u and not x?

Note, this is what we refer to as *minimum control energy*; the energy where only the control signals are constrained

In [6]:
system = 'continuous'  # choose time system
A_norm = matrix_normalization(A, system=system, c=1)  # normalize A
B = np.eye(n_nodes)  # uniform full control set
T = 1  # time horizon
xr = np.zeros(n_nodes)  # x and u constrained toward zero activity
#  <-- NOTE REMOVAL OF RHO
S = np.zeros((n_nodes, n_nodes))  # x is *not* constrained  <-- NOTE CHANGE

# get the state trajectory (x) and the control inputs (u)
x, u, n_err = get_control_inputs(A_norm=A_norm, T=T, B=B, x0=x0, xf=xf, system=system, xr=xr, S=S)  # <-- NOTE REMOVAL OF RHO
node_energy = integrate_u(u)
energy = np.sum(node_energy)

print('energy: {0:.2f}'.format(energy))

energy: 2570.65


### You want to constrain $x$ and $u$ toward the target state instead of toward zero?

In [7]:
system = 'continuous'  # choose time system
A_norm = matrix_normalization(A, system=system, c=1)  # normalize A
B = np.eye(n_nodes)  # uniform full control set
T = 1  # time horizon
xr = xf  # x and u constrained toward target start  <-- NOTE CHANGE
rho = 1  # x and u constrained equally
S = np.eye(n_nodes)  # x is constrained over all nodes

# get the state trajectory (x) and the control inputs (u)
x, u, n_err = get_control_inputs(A_norm=A_norm, T=T, B=B, x0=x0, xf=xf, system=system, xr=xr, rho=rho, S=S)
node_energy = integrate_u(u)
energy = np.sum(node_energy)

print('energy: {0:.2f}'.format(energy))

energy: 2607.39


### You want to use a discrete time system instead?

Note, time horizon must be >1 for discrete time systems. In a discrete system, time is broken into
discrete steps of 1, which means T=1 would be only 1 time step.

In [8]:
system = 'discrete'  # choose time system  <-- NOTE CHANGE
A_norm = matrix_normalization(A, system=system, c=1)  # normalize A
B = np.eye(n_nodes)  # uniform full control set
T = 10  # time horizon  <-- NOTE CHANGE. TIME HORIZON MUST BE >1 FOR DISCRETE TIME SYSTEMS
xr = np.zeros(n_nodes)  # x and u constrained toward zero activity
rho = 1  # x and u constrained equally
S = np.eye(n_nodes)  # x is constrained over all nodes

# get the state trajectory (x) and the control inputs (u)
x, u, n_err = get_control_inputs(A_norm=A_norm, T=T, B=B, x0=x0, xf=xf, system=system, xr=xr, rho=rho, S=S)
node_energy = integrate_u(u)
energy = np.sum(node_energy)

print('energy: {0:.2f}'.format(energy))

energy: 0.51
